<subject>
Homework How-To
</subject>

<details>

**Naming Conventions**

When naming your files for upload, you must follow the format below:

    <uni>_<assignment>_<details [optional]>.<extension>

For example, if I needed to hand in HW 0, any of the below formats would be sufficient for a file name:

- pl2648_hw0.txt
- pl2648_hw0.sh
- pl2648_hw0_all_in_one.txt
- Pl2648_hw0_bash_program.sh

This naming format allows for autograding of all assignments. If your files are not named with this format, you should expect a grade of zero for the assignment.

**Grading**

Possible points on late homeworks are deducted by 50% for each day they are late. For example, if you get 80% of the total possible credits on a homework but hand in that homework a day late, you would get 40%. Assignments two days late get zero points.

Once solutions are posted and graded assignments are handed back, students have 1 week to bring their grading discrepancies to a CA for consideration of possible grading errors. 

**Getting Help**

Asking for help is a great way to increase your chance of success. However there are some rules. When asking for help (especially from a fellow student), *you can show your helper your code but you can not view theirs*. You work needs to be your own. 

If you need to reach out to a CA or the Professor for help, please do so via Piazza and not via email. Answers given via Piazza will help you as well as other students. Thus, emails will always have a lower priority for response than Piazza questions. If you do email the professor or TA, please make a note of what section you are in. This helps us identify you in courseworks faster.
</details>

For the following questions, assume you have the following tables of information in the database `nmh`. 

- table name: `exhibits`

id (INT) | name (VARCHAR) | start_date (DATE) | end_date (DATE) | curator_id (INT) [FK to curator.id]
-|-|-|-|-
3| Free The Fishes | 2018-01-01 | 2018-06-30 | 5
17| Space, What Lies Above | 2018-02-01 | 2018-05-30 | 11
23| Bears Bears Bears | 2018-02-14 | 2018-02-24 | 5
46| Humans? Aliens? | 2019-03-14 | 2019-10-21 | 11


- table name: `curators`

id (INT) | name (VARCHAR) | bio (TEXT)
-|-|-
5| Rebecca Votea | Esteemed naturalist
11| Simon Strauss | Space man
71| Rick Sanchez | Grandfather


SQL solutions will be graded on their simplicity as well as on whether or not they would return the correct answer from the database.

**Q**: Write a function that returns a string containing the SQL necessary to create two tables above.

These tables are necessary for future problems.

[0 points]

In [ ]:
def create_tables_sql() -> str:
    return 'CREATE TABLE exhibits (id INTEGER PRIMARY KEY, name VARCHAR(255), start_date DATE, end_date DATE, curator_id INTEGER); CREATE TABLE curators (id INTEGER PRIMARY KEY, name VARCHAR(255), bio TEXT);'

In [ ]:
### BEGIN TESTS
import sqlite3
conn = sqlite3.connect(':memory:')

c = conn.cursor()

c.executescript(create_tables_sql())

conn.commit()
### END TESTS

**Q**: Write a function that returns a string containing the SQL necessary to add the data from the two tables above.

This data is necessary for future problems.

[0 points]

In [ ]:
def insert_data_sql() -> str:
    return "INSERT INTO exhibits VALUES (3, 'Free The Fishes', '2018-01-01', '2018-06-30',5),(17, 'Space, What Lies Above','2018-02-01','2018-05-30',11),(23, 'Bears Bears Bears','2018-02-14','2018-02-24',5),(46,'Humans? Aliens?','2019-03-14','2019-10-21',11); INSERT INTO curators VALUES (5,'Rebecca Votea','Esteemed naturalist'),(11,'Simon Strauss','Space man'),(71,'Rick Sanchez','Grandfather');"

In [ ]:
### BEGIN TESTS
c.executescript(insert_data_sql())
conn.commit()
### END TESTS

**Q**: Create a function that will return the SQL to find all the names of all the curators ordered by their id. IDs should be in descending order. Do not use an alias for the table name.

[1 point]

In [ ]:
def get_sql():
    return 'SELECT name FROM curators ORDER BY id DESC'

In [ ]:
### BEGIN TESTS
c.execute(get_sql())
result = c.fetchall()
assert result == [('Rick Sanchez',), ('Simon Strauss',), ('Rebecca Votea',)]
### BEGIN TESTS

**Q**: Create a function that will return the SQL to find all the names of all exhibts and the names of their associated curators. If an exhibit does not have a curator, the exhibit should not be in the result. Even if a curator does not have an exhibit, the curator should be present in the final result. The first letter of the table name should be used as its alias.

For example,

```
Rebecca Votea, Free The Fishes
Rebecca Votea, Bears Bears Bears
Rick Sanchez, 
...
```

[1 point]

In [ ]:
def get_sql():
    return "SELECT c.name, e.name FROM curators c LEFT JOIN exhibits e ON e.curator_id=c.id;"

In [ ]:
### BEGIN TESTS
c.execute(get_sql())
result = set(c.fetchall())
expected = {
    ('Rebecca Votea', 'Bears Bears Bears'),
    ('Rebecca Votea', 'Free The Fishes'),
    ('Rick Sanchez', None),
    ('Simon Strauss', 'Humans? Aliens?'),
    ('Simon Strauss', 'Space, What Lies Above')
}
assert result == expected
### BEGIN TESTS

**Q**: Create a function that will return the SQL to find the number of exhibits for all curators with a name that starts with "R" (case-insensitive). The output of running the SQL should be two fields (name, count). 

Even if a curator does not have an exhibit, the curator should be present in the final result. The first letter of the table name should be used as its alias. You can not assume curators names are distinct.

[1 point]

In [ ]:
def get_sql():
    return "SELECT c.name, COUNT(e.name) FROM curators c LEFT JOIN exhibits e ON e.curator_id=c.id WHERE c.name LIKE 'R%' GROUP BY c.id;"

In [ ]:
### BEGIN TESTS
c.execute(get_sql())
result = set(c.fetchall())
expected = {
    ('Rebecca Votea', 2), 
    ('Rick Sanchez', 0),
}
assert result == expected
### BEGIN TESTS

**Q**: Create a function that will return the SQL to find the exhibits with overlapping runs (overlapping intervals of time bound by start date and end date (inclusive)). The result should include only one instance of each overlapping pair. The exhibit with the lower valued ID should be listed as `exhibit1` in the result and the other with `exhibit2`.

The result should be:

```
        exhibit1        |        exhibit2        
------------------------+------------------------
 Free The Fishes        | Space, What Lies Above
 Free The Fishes        | Bears Bears Bears
 Space, What Lies Above | Bears Bears Bears
```

The first letter of the table name should be used as its alias. If multiple instances of the same table are necessary, append a number to the end of the alias. The number sequence appended to the alias should start at 1.

[1 point]

In [ ]:
def get_sql():
    return "SELECT e1.name as exhibit1,e2.name as exhibit2 FROM exhibits e1,exhibits e2 WHERE e1.id <> e2.id AND (e2.start_date>=e1.start_date AND e2.end_date<=e1.end_date);"

In [ ]:
### BEGIN TESTS
c.execute(get_sql())
result = set(c.fetchall())
result = tuple(map(set, result))

expected = (
    {'Bears Bears Bears', 'Space, What Lies Above'},
    {'Bears Bears Bears', 'Free The Fishes'}, 
    {'Free The Fishes', 'Space, What Lies Above'}
)

for set_ in expected:
    assert set_ in result

assert len(result) == 3
### BEGIN TESTS